In [8]:
from rdkit import Chem
from rdkit.Chem import AllChem, rdDistGeom
from morfeus import XTB
import numpy as np

def get_molecules(smiles_list):
    molecule_descriptors = []
    for smiles in smiles_list:
        mol = Chem.MolFromSmiles(smiles)
        mol = Chem.AddHs(mol)
        
        params = rdDistGeom.ETKDG()
        params.numThreads = 0
        AllChem.EmbedMultipleConfs(mol, numConfs=10, params=params)
        
        descriptor_sum = 0
        for conf_id in range(mol.GetNumConformers()):
            # Call xTB to calculate electrophilicity for the conformer
            electrophilicity = calculate_xtb_property(mol, conf_id, "electrophilicity")
            descriptor_sum += electrophilicity
        
        # Average
        molecule_descriptors.append((smiles, descriptor_sum / mol.GetNumConformers()))
    
    # Ranking
    molecule_descriptors.sort(key=lambda x: x[1], reverse=True)
    return molecule_descriptors

def calculate_xtb_property(mol, conf_id, property_type):
    num_atoms = mol.GetNumAtoms()
    xyz_lines = [str(num_atoms)]
    xyz_lines.append(Chem.MolToSmiles(mol))
    conf = mol.GetConformer(conf_id)
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        pos = conf.GetAtomPosition(atom.GetIdx())
        xyz_lines.append(f"{symbol} {pos.x:.4f} {pos.y:.4f} {pos.z:.4f}")
    xyz_data = "\n".join(xyz_lines)
    
    xtb = XTB(elements=[atom.GetSymbol() for atom in mol.GetAtoms()], 
               coordinates=np.array([conf.GetAtomPosition(i) for i in range(num_atoms)]))
    if property_type == "electrophilicity":
        property_value = xtb.get_global_descriptor("electrophilicity", corrected=True)
    elif property_type == "nucleophilicity":
        property_value = xtb.get_global_descriptor("nucleophilicity", corrected=True)
    return property_value

if __name__ == "__main__":
    user_input = input("Enter a list of SMILES strings separated by commas: ")
    smiles_list_input = user_input.split(',')
    smiles_list_input = [smiles.strip() for smiles in smiles_list_input]

    ranked_molecules = get_molecules(smiles_list_input)


    print("Ranked Molecules based on average global descriptor (electrophilicity)")
    for i, (molecule, descriptor) in enumerate(ranked_molecules, start=1):
        print(f"{i}. {molecule} - Average Electrophilicity: {descriptor}")


Enter a list of SMILES strings separated by commas:  CNO, CNC, CN, Nc1ccccc1


Ranked Molecules based on average global descriptor (electrophilicity)
1. Nc1ccccc1 - Average Electrophilicity: 0.4233115448192623
2. CNC - Average Electrophilicity: 0.007995701288634752
3. CNO - Average Electrophilicity: 0.002327335853380086
4. CN - Average Electrophilicity: 0.0008408780219761817
